In [1]:
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
def analyze_read(html_doc):
    soup = BeautifulSoup(html_doc, 'html.parser')
    return soup.get_text()

In [3]:
df = pd.read_excel("CLIC_Content_List.xlsx", usecols = ['title','full_path','topic','table','content']).dropna()

In [8]:
topics = df['topic'].unique()
topics

array(['landlordTenant', 'defamation', 'insurance', 'personalDataPrivacy',
       'intellectualProperty', 'ADR', 'antiDiscrimination', 'bankcruptcy',
       'businessAndCommerce', 'civilCase', 'trafficOffences',
       'competitionLaw', 'consumerComplaints',
       'diyResidentialTenancyAgreement', 'employmentDisputes',
       'enduringPowersOfAttorney', 'family', 'freedomOfAssembly',
       'hkLegalSystem', 'immigration', 'legalAid',
       'maintenanceAndSafetyOfProperty', 'medicalTreatmentConsent',
       'medicalNegligence', 'personalInjuries', 'policeAndCrime',
       'probate', 'protectionForInvestors', 'redevelopmentAndAcquisition',
       'saleAndPurchaseOfProperty', 'sexualOffences', 'taxation'],
      dtype=object)

In [ ]:
with open ("TOPICS.txt", "w") as f:
    for topic in topics:
        f.write(topic + '\n')

In [17]:

df['parsed_content'] = df['content'].apply(analyze_read)
print(df.head())

                                               title  \
0                                        Sub-letting   
1                     Repair/maintenance obligations   
2  1. In general, who shall be responsible for ma...   
3  1. I received a letter from a bank claiming to...   
4  Landlord sells the property with existing tenancy   

                                           full_path           topic  table  \
0  https://clic.org.hk/en/topics/landlord_tenant/...  landlordTenant    0.0   
1  https://clic.org.hk/en/topics/landlord_tenant/...  landlordTenant    0.0   
2  https://clic.org.hk/en/topics/landlord_tenant/...  landlordTenant    0.0   
3  https://clic.org.hk/en/topics/landlord_tenant/...  landlordTenant    0.0   
4  https://clic.org.hk/en/topics/landlord_tenant/...  landlordTenant    0.0   

                                             content  \
0  <h2>VI. Sub-letting</h2> <p><strong>The Nature...   
1  <h2>VIII. Repair/maintenance obligations</h2> ...   
2  <h2>1. In general